# 基于深度学习的 MZM 导频法闭环控制（数据集生成 / 训练 / 推理）

本 Notebook 的目标：
- 只使用 PD 输出的导频 1f/2f **功率(dBm)**（不使用 DC 光功率/不依赖高速 RF 可观测）。
- 目标角度范围：**0–180°**（对应单个周期半波）。
- 将 **数据集生成 → 训练 → 推理/回放** 拆开，便于复用先前产物。

In [ ]:
from __future__ import annotations

import importlib
import numpy as np

# Reusable pipeline module (reload to pick up edits)
import mzm.dither_controller as dc
dc = importlib.reload(dc)

DATASET_PATH = 'artifacts/dither_dataset_dbm_hist.npz'
MODEL_PATH = 'artifacts/dither_policy_dbm_hist.pt'

In [ ]:
import torch
import sys

print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")

if torch.cuda.is_available():
    print(f"✅ CUDA is available. Device: {torch.cuda.get_device_name(0)}")
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    print("✅ MPS (Metal Performance Shaders) is available on macOS.")
else:
    print("ℹ️ No GPU acceleration detected. Using CPU.")

## 数据集生成

数据集会保存到 `artifacts/`，下次可直接加载复用。

In [ ]:
device_params = dc.DeviceParams()
dither_params = dc.DitherParams()

# Generate (or overwrite) dataset artifact
# 增大样本量以覆盖 0 度附近的物理死区（灵敏度低区域）
ds0 = dc.generate_dataset_dbm_hist(
    device_params=device_params,
    dither_params=dither_params,
    n_samples=60000,  # Increased from 20000 to 60000 to improve edge/null-point accuracy
    seed=0,
    teacher_gain=0.5,
    max_step_V=0.2,
    accel='auto',
    torch_batch=128,
 )
dc.save_dataset(ds0, DATASET_PATH)
print('saved dataset:', DATASET_PATH)

# If you used CUDA/MPS, PyTorch may keep a caching allocator reserve in VRAM.
# This is normal (not a leak). If you want to release it back to the OS:
import gc
import torch
del ds0
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
if hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    torch.mps.empty_cache()

ds = dc.load_dataset(DATASET_PATH)
print('Xn:', ds['Xn'].shape, 'y:', ds['y'].shape)
print('mu:', ds['mu'].shape, 'sigma:', ds['sigma'].shape)
print('device_params:', ds['device_params'])
print('dither_params:', ds['dither_params'])

## 训练

训练输出保存到 `artifacts/dither_policy_dbm_hist.pt`，可重复加载复用。

In [ ]:
import importlib
import mzm.dither_controller as dc
dc = importlib.reload(dc)

ds = dc.load_dataset(DATASET_PATH)

# Note: accel='cpu' is often faster for small models due to lower latency
# We have optimized the training loop to remove DataLoader overhead.
# 调整训练参数以避免过拟合噪声，并提高在死区附近的泛化能力
model = dc.train_policy(
    Xn=ds['Xn'],
    y=ds['y'],
    epochs=5000,   # 降低轮数，防止对 0 度附近的噪声过拟合 (was 10000)
    batch=1024,    # 增大 Batch Size，使梯度更新更平滑 (was 256)
    lr=5e-4,       # 降低学习率，精细搜索最优解 (was 1e-3)
    hidden=128,    # 增加模型容量，以拟合非线性极值点特征 (was 64)
    depth=3,
    seed=0,
    accel='auto',
 )

dc.save_model(
    model=model,
    mu=ds['mu'],
    sigma=ds['sigma'],
    device_params=ds['device_params'],
    dither_params=ds['dither_params'],
    path=MODEL_PATH,
 )
print('saved model:', MODEL_PATH)

## 推理/回放（单目标角度 + 逐轮过程 + 前后光谱对比）

- 选择一个 `target_deg` 作为锁定目标；
- 展示每一轮迭代的观测与动作（`p1/p2`、差分、`ΔV`、`V`、`θ`、误差）；
- 在初始偏置与终止偏置分别绘制一次**光谱/电谱**用于前后对比。

In [ ]:
import matplotlib.pyplot as plt

from mzm.model import simulate_mzm, bias_to_theta_rad
from mzm.plot import plot_optical_spectrum_osa, plot_electrical_spectrum

%config InlineBackend.figure_format = 'retina'

model, meta = dc.load_model(MODEL_PATH)

# Choose a single target working point (deg)
target_deg = 0.0

# Initial bias: random by default.
# - Set init_seed to an integer for reproducible runs (e.g., 1).
# - Keep it as None to re-sample a different V_init each run.
init_seed = None
rng = np.random.default_rng(init_seed)
V_init = float(rng.uniform(0.0, float(meta['device_params'].Vpi_DC)))

theta_init_deg = float(np.rad2deg(bias_to_theta_rad(V_init, Vpi_DC=float(meta['device_params'].Vpi_DC))))

print('target_deg     :', target_deg)
print('init V (V)     :', V_init)
print('init theta (deg):', theta_init_deg)

# --- (A) Pre-control spectra at initial bias ---
print('\n[pre] spectra at V_init')
sim_pre = simulate_mzm(
    V_bias=V_init,
    Vpi_RF=float(meta['device_params'].Vpi_DC),
    Vpi_DC=float(meta['device_params'].Vpi_DC),
    ER_dB=float(meta['device_params'].ER_dB),
    IL_dB=float(meta['device_params'].IL_dB),
    Pin_dBm=float(meta['device_params'].Pin_dBm),
    Responsivity=float(meta['device_params'].Responsivity),
    R_load=float(meta['device_params'].R_load),
    # RF settings used only for spectrum illustration
    f_rf=1e9,
    V_rf_amp=0.2,
    Fs=100e9,
    T_total=5e-6,
 )
print('pre  P_pd_avg_dBm:', sim_pre.P_pd_avg_dBm)
plot_optical_spectrum_osa(sim_pre, f_rf_hz=1e9)
plot_electrical_spectrum(sim_pre, f_rf_hz=1e9)

# --- (B) Closed-loop rollout (dBm-only + history) ---
steps = 60
r = dc.rollout_dbm_hist(
    model=model,
    mu=meta['mu'],
    sigma=meta['sigma'],
    device_params=meta['device_params'],
    dither_params=meta['dither_params'],
    theta_target_deg=float(target_deg),
    V_init=V_init,
    steps=steps,
    accel='cpu',
 )

V_final = float(r['V'][-1])
final_err = float(r['err_deg'][-1])
print('\n[rollout] final V:', V_final)
print('[rollout] final wrapped err (deg):', f"{final_err:+.2f}")

# Print per-iteration trace
print('\nPer-iteration trace:')
print(' iter |    V(V) |  theta(deg) |  err(deg) |   dV(V) |  p1(dBm) |  p2(dBm) | dp1(dB) | dp2(dB)')
for k in range(steps):
    print(
        f"{k+1:5d} |"
        f" {r['V'][k]:7.4f} |"
        f" {r['theta_deg'][k]:10.3f} |"
        f" {r['err_deg'][k]:8.3f} |"
        f" {r['dv'][k]:7.4f} |"
        f" {r['p1_dBm'][k]:8.2f} |"
        f" {r['p2_dBm'][k]:8.2f} |"
        f" {r['dp1_dBm'][k]:7.2f} |"
        f" {r['dp2_dBm'][k]:7.2f}"
    )

# Plot convergence curve
plt.figure(figsize=(8, 4))
plt.plot(r['err_deg'], label=f"target {target_deg:.0f}°")
plt.axhline(0.0, color='k', linewidth=1)
plt.xlabel('Iteration')
plt.ylabel('Phase Error (deg, wrapped)')
plt.title('Closed-loop convergence (dBm-only + history)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# --- (C) Post-control spectra at final bias ---
print('\n[post] spectra at V_final')
sim_post = simulate_mzm(
    V_bias=V_final,
    Vpi_RF=float(meta['device_params'].Vpi_DC),
    Vpi_DC=float(meta['device_params'].Vpi_DC),
    ER_dB=float(meta['device_params'].ER_dB),
    IL_dB=float(meta['device_params'].IL_dB),
    Pin_dBm=float(meta['device_params'].Pin_dBm),
    Responsivity=float(meta['device_params'].Responsivity),
    R_load=float(meta['device_params'].R_load),
    f_rf=1e9,
    V_rf_amp=0.2,
    Fs=100e9,
    T_total=5e-6,
 )
print('post P_pd_avg_dBm:', sim_post.P_pd_avg_dBm)
plot_optical_spectrum_osa(sim_post, f_rf_hz=1e9)
plot_electrical_spectrum(sim_post, f_rf_hz=1e9)

## 模型评估与对比

扫描 0-180° 目标角度，每个角度重复推理 50 次（根据显存大小调整推理次数）。
- **可视化**：绘制平均误差曲线，并使用阴影区域表示方差（标准差范围），类似 K 线图的概念展示波动情况。
- **缓存**：自动加载历史最佳模型的评估结果（`artifacts/dither_policy_dbm_hist_best_eval.npz`），避免重复计算。

In [ ]:
import os
import shutil
import hashlib
import numpy as np
import matplotlib.pyplot as plt
import torch
import importlib
import gc
import mzm.dither_controller as dc

# Reload to ensure we have the new batch function
dc = importlib.reload(dc)

# Paths
BEST_MODEL_PATH = 'artifacts/dither_policy_dbm_hist_best.pt'
BEST_EVAL_PATH = 'artifacts/dither_policy_dbm_hist_best_eval.npz'

def get_file_hash(filepath):
    """Calculate MD5 hash of a file."""
    if not os.path.exists(filepath):
        return None
    with open(filepath, "rb") as f:
        return hashlib.md5(f.read()).hexdigest()

def evaluate_model(model_path, label, steps=60, n_repeats=50):
    print(f"Evaluating model: {label} ...")
    
    # Load model
    try:
        loaded_model, loaded_meta = dc.load_model(model_path)
    except FileNotFoundError:
        print(f"Model not found: {model_path}")
        return None

    targets = np.arange(0, 181, 1)
    
    # Use the params from the loaded model
    device_params = loaded_meta['device_params']
    dither_params = loaded_meta['dither_params']
    mu = loaded_meta['mu']
    sigma = loaded_meta['sigma']
    
    init_seed = None
    rng = np.random.default_rng(init_seed)
    
    # Prepare Batch Inputs
    # Repeat targets: [0, 0, ..., 1, 1, ..., 180, 180, ...]
    targets_repeated = np.repeat(targets, n_repeats)
    n_total = len(targets_repeated)
    
    # Random V_init for each simulation
    V_init_batch = rng.uniform(0.0, float(device_params.Vpi_DC), size=n_total)
    
    # print(f"Running batch inference for {n_total} trajectories...")
    
    # Run batch rollout
    # Note: We use 'auto' here because batch size is large (~1800), so GPU/MPS acceleration is beneficial.
    r = dc.rollout_dbm_hist_batch(
        model=loaded_model,
        mu=mu,
        sigma=sigma,
        device_params=device_params,
        dither_params=dither_params,
        theta_target_deg=targets_repeated,
        V_init=V_init_batch,
        steps=steps,
        accel='auto' 
    )
    
    # Extract final errors (absolute)
    # r['err_deg'] shape is (N, steps)
    final_errs = np.abs(r['err_deg'][:, -1])
    
    # Cleanup memory immediately
    del r
    del loaded_model
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    if hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
        torch.mps.empty_cache()
    
    # Reshape to (n_targets, n_repeats)
    final_errs_reshaped = final_errs.reshape(len(targets), n_repeats)
    
    # Calculate stats
    means = np.mean(final_errs_reshaped, axis=1)
    stds = np.std(final_errs_reshaped, axis=1)
    mins = np.min(final_errs_reshaped, axis=1)
    maxs = np.max(final_errs_reshaped, axis=1)
        
    return {
        'targets': targets,
        'mean': means,
        'std': stds,
        'min': mins,
        'max': maxs,
        'label': label
    }

def plot_eval_results(current_stats, best_stats=None):
    plt.figure(figsize=(12, 6))
    
    # Helper to plot one set of stats
    def plot_one(stats, color, label_prefix):
        t = stats['targets']
        m = stats['mean']
        s = stats['std']
        
        # Plot Mean
        plt.plot(t, m, color=color, label=f"{label_prefix} Mean MAE", linewidth=2)
        
        # Plot Std Dev (Variance representation)
        # Clip lower bound to 0 because absolute error cannot be negative
        lower_bound = np.maximum(m - s, 0)
        plt.fill_between(t, lower_bound, m + s, color=color, alpha=0.2, label=f"{label_prefix} ±1 Std (Variance)")

    if current_stats:
        plot_one(current_stats, 'blue', 'Current')
        print(f"Current Model Overall Mean MAE: {np.mean(current_stats['mean']):.4f} deg")

    if best_stats:
        plot_one(best_stats, 'orange', 'Best')
        print(f"Best Model Overall Mean MAE: {np.mean(best_stats['mean']):.4f} deg")

    plt.xlabel('Target Angle (deg)')
    plt.ylabel('Absolute Error (deg)')
    plt.title('Model Evaluation: Error Distribution vs Target Angle (0-180°)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

# 1. Evaluate Current Model
current_stats = evaluate_model(MODEL_PATH, "Current Model")

# 2. Load Best Stats (if available) or Evaluate Best Model (if pt exists but npz doesn't)
best_stats = None

if os.path.exists(BEST_EVAL_PATH):
    print(f"Loading cached best evaluation from {BEST_EVAL_PATH}")
    data = np.load(BEST_EVAL_PATH)
    best_stats = {k: data[k] for k in data.files}
    best_stats['label'] = "Best Model (Cached)"
elif os.path.exists(BEST_MODEL_PATH):
    print("Cached evaluation not found, re-evaluating best model...")
    best_stats = evaluate_model(BEST_MODEL_PATH, "Best Model")

# 3. Plot
plot_eval_results(current_stats, best_stats)

# 4. Automatic Update Logic
if current_stats:
    curr_mae = np.mean(current_stats['mean'])
    
    if best_stats:
        best_mae = np.mean(best_stats['mean'])
        print(f"\nComparison: Current ({curr_mae:.4f}) vs Best ({best_mae:.4f})")
        
        if curr_mae < best_mae:
            # Check if models are identical
            curr_hash = get_file_hash(MODEL_PATH)
            best_hash = get_file_hash(BEST_MODEL_PATH)
            
            if curr_hash == best_hash:
                print("⚠️ Current model file is identical to the best, but metrics improved. Updating best evaluation cache only.")
                np.savez(
                    BEST_EVAL_PATH,
                    targets=current_stats['targets'],
                    mean=current_stats['mean'],
                    std=current_stats['std'],
                    min=current_stats['min'],
                    max=current_stats['max']
                )
                print(f"✅ Updated best stats: {BEST_EVAL_PATH}")
            else:
                print("🚀 Current model is BETTER! Updating best model...")
                shutil.copy2(MODEL_PATH, BEST_MODEL_PATH)
                np.savez(
                    BEST_EVAL_PATH,
                    targets=current_stats['targets'],
                    mean=current_stats['mean'],
                    std=current_stats['std'],
                    min=current_stats['min'],
                    max=current_stats['max']
                )
                print(f"✅ Updated best model: {BEST_MODEL_PATH}")
                print(f"✅ Updated best stats: {BEST_EVAL_PATH}")
        else:
            print("📉 Current model is NOT better than the best model. Keep training!")
    else:
        # No best model exists yet
        print("🚀 No best model found. Setting current model as best...")
        shutil.copy2(MODEL_PATH, BEST_MODEL_PATH)
        np.savez(
            BEST_EVAL_PATH,
            targets=current_stats['targets'],
            mean=current_stats['mean'],
            std=current_stats['std'],
            min=current_stats['min'],
            max=current_stats['max']
        )
        print(f"✅ Created best model: {BEST_MODEL_PATH}")